# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fd3ecb61d60>
├── 'a' --> tensor([[-1.5201,  0.3167, -0.5673],
│                   [-0.4560, -0.7056,  1.3035]])
└── 'x' --> <FastTreeValue 0x7fd3ecbcd130>
    └── 'c' --> tensor([[ 0.1531,  0.4366, -0.5599, -0.4469],
                        [ 1.2017,  1.1118, -1.0696,  0.1021],
                        [-1.2450, -0.8708, -0.1355, -2.3264]])

In [4]:
t.a

tensor([[-1.5201,  0.3167, -0.5673],
        [-0.4560, -0.7056,  1.3035]])

In [5]:
%timeit t.a

63.7 ns ± 0.035 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fd3ecb61d60>
├── 'a' --> tensor([[ 1.1439,  0.9304,  0.3507],
│                   [-0.4510,  0.3112,  0.1476]])
└── 'x' --> <FastTreeValue 0x7fd3ecbcd130>
    └── 'c' --> tensor([[ 0.1531,  0.4366, -0.5599, -0.4469],
                        [ 1.2017,  1.1118, -1.0696,  0.1021],
                        [-1.2450, -0.8708, -0.1355, -2.3264]])

In [7]:
%timeit t.a = new_value

66.4 ns ± 0.12 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-1.5201,  0.3167, -0.5673],
               [-0.4560, -0.7056,  1.3035]]),
    x: Batch(
           c: tensor([[ 0.1531,  0.4366, -0.5599, -0.4469],
                      [ 1.2017,  1.1118, -1.0696,  0.1021],
                      [-1.2450, -0.8708, -0.1355, -2.3264]]),
       ),
)

In [10]:
b.a

tensor([[-1.5201,  0.3167, -0.5673],
        [-0.4560, -0.7056,  1.3035]])

In [11]:
%timeit b.a

60.2 ns ± 0.0139 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 1.1971,  0.3646,  0.5112],
               [ 0.4454, -1.4249, -1.6989]]),
    x: Batch(
           c: tensor([[ 0.1531,  0.4366, -0.5599, -0.4469],
                      [ 1.2017,  1.1118, -1.0696,  0.1021],
                      [-1.2450, -0.8708, -0.1355, -2.3264]]),
       ),
)

In [13]:
%timeit b.a = new_value

498 ns ± 0.101 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

825 ns ± 1.93 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.9 µs ± 19.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

145 µs ± 1.49 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

148 µs ± 784 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fd3ecb61a00>
├── 'a' --> tensor([[[-1.5201,  0.3167, -0.5673],
│                    [-0.4560, -0.7056,  1.3035]],
│           
│                   [[-1.5201,  0.3167, -0.5673],
│                    [-0.4560, -0.7056,  1.3035]],
│           
│                   [[-1.5201,  0.3167, -0.5673],
│                    [-0.4560, -0.7056,  1.3035]],
│           
│                   [[-1.5201,  0.3167, -0.5673],
│                    [-0.4560, -0.7056,  1.3035]],
│           
│                   [[-1.5201,  0.3167, -0.5673],
│                    [-0.4560, -0.7056,  1.3035]],
│           
│                   [[-1.5201,  0.3167, -0.5673],
│                    [-0.4560, -0.7056,  1.3035]],
│           
│                   [[-1.5201,  0.3167, -0.5673],
│                    [-0.4560, -0.7056,  1.3035]],
│           
│                   [[-1.5201,  0.3167, -0.5673],
│                    [-0.4560, -0.7056,  1.3035]]])
└── 'x' --> <FastTreeValue 0x7fd3ecb61e20>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.7 µs ± 99.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fd335e6d8b0>
├── 'a' --> tensor([[-1.5201,  0.3167, -0.5673],
│                   [-0.4560, -0.7056,  1.3035],
│                   [-1.5201,  0.3167, -0.5673],
│                   [-0.4560, -0.7056,  1.3035],
│                   [-1.5201,  0.3167, -0.5673],
│                   [-0.4560, -0.7056,  1.3035],
│                   [-1.5201,  0.3167, -0.5673],
│                   [-0.4560, -0.7056,  1.3035],
│                   [-1.5201,  0.3167, -0.5673],
│                   [-0.4560, -0.7056,  1.3035],
│                   [-1.5201,  0.3167, -0.5673],
│                   [-0.4560, -0.7056,  1.3035],
│                   [-1.5201,  0.3167, -0.5673],
│                   [-0.4560, -0.7056,  1.3035],
│                   [-1.5201,  0.3167, -0.5673],
│                   [-0.4560, -0.7056,  1.3035]])
└── 'x' --> <FastTreeValue 0x7fd335e6dee0>
    └── 'c' --> tensor([[ 0.1531,  0.4366, -0.5599, -0.4469],
                        [ 1.2017,  1.1118, -1.0696,  0.1021],
                 

In [23]:
%timeit t_cat(trees)

30.2 µs ± 64.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

57.5 µs ± 62.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.1531,  0.4366, -0.5599, -0.4469],
                       [ 1.2017,  1.1118, -1.0696,  0.1021],
                       [-1.2450, -0.8708, -0.1355, -2.3264]],
              
                      [[ 0.1531,  0.4366, -0.5599, -0.4469],
                       [ 1.2017,  1.1118, -1.0696,  0.1021],
                       [-1.2450, -0.8708, -0.1355, -2.3264]],
              
                      [[ 0.1531,  0.4366, -0.5599, -0.4469],
                       [ 1.2017,  1.1118, -1.0696,  0.1021],
                       [-1.2450, -0.8708, -0.1355, -2.3264]],
              
                      [[ 0.1531,  0.4366, -0.5599, -0.4469],
                       [ 1.2017,  1.1118, -1.0696,  0.1021],
                       [-1.2450, -0.8708, -0.1355, -2.3264]],
              
                      [[ 0.1531,  0.4366, -0.5599, -0.4469],
                       [ 1.2017,  1.1118, -1.0696,  0.1021],
                       [-1.2450, -0.8708, -0.1355, -2.3264]],

In [26]:
%timeit Batch.stack(batches)

79.2 µs ± 129 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.1531,  0.4366, -0.5599, -0.4469],
                      [ 1.2017,  1.1118, -1.0696,  0.1021],
                      [-1.2450, -0.8708, -0.1355, -2.3264],
                      [ 0.1531,  0.4366, -0.5599, -0.4469],
                      [ 1.2017,  1.1118, -1.0696,  0.1021],
                      [-1.2450, -0.8708, -0.1355, -2.3264],
                      [ 0.1531,  0.4366, -0.5599, -0.4469],
                      [ 1.2017,  1.1118, -1.0696,  0.1021],
                      [-1.2450, -0.8708, -0.1355, -2.3264],
                      [ 0.1531,  0.4366, -0.5599, -0.4469],
                      [ 1.2017,  1.1118, -1.0696,  0.1021],
                      [-1.2450, -0.8708, -0.1355, -2.3264],
                      [ 0.1531,  0.4366, -0.5599, -0.4469],
                      [ 1.2017,  1.1118, -1.0696,  0.1021],
                      [-1.2450, -0.8708, -0.1355, -2.3264],
                      [ 0.1531,  0.4366, -0.5599, -0.4469],
                   

In [28]:
%timeit Batch.cat(batches)

147 µs ± 499 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

338 µs ± 3.22 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
